In [1]:
%load_ext autoreload
%autoreload 2

In [66]:
import predict
import glob
import skimage
import sklearn
import numpy as np

def raw_symbols_to_input(irawsymbols, image_size):
    iprocessed = skimage.filters.gaussian(irawsymbols, 1)
    X = skimage.transform.resize(iprocessed, output_shape=(len(iprocessed), image_size[0], image_size[1]))
    return X

y = []
X = []
datadir = 'data/MYDATA/train/*.png'
for name in glob.glob(datadir):
    symbol = name.split('_')[1].replace(".png","")
    ilabels, irawsymbols = predict.file_to_raw_symbols(name, single_symbol=True)
    inputsymbols = raw_symbols_to_input(irawsymbols, (50, 50))    
    X.append(inputsymbols.flatten())
    y.append(symbol)
    
X = np.array(X)
X = np.squeeze(X)
sklearn.externals.joblib.dump(X, 'data/MYDATA/images_symbols.p')
sklearn.externals.joblib.dump(y, 'data/MYDATA/symbols.p')

['data/MYDATA/symbols.p']

In [87]:
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.externals import joblib
y = joblib.load('data/MYDATA/symbols.p')
X = joblib.load('data/MYDATA/images_symbols.p')

In [88]:
def my_transform(X):
    images = X.reshape((1330, 50, 50))
    hogs = []
    for image in images:
        fd = skimage.feature.hog(image)
        hogs.append(fd)
    hogs = np.array(hogs)
    return np.array(hogs)

In [107]:
Xt = my_transform(X)

from sklearn import ensemble

estimators = [(('reduce_dim'), sklearn.decomposition.PCA()),
             (('clf', sklearn.neural_network.MLPClassifier()))]
pipe = sklearn.pipeline.Pipeline(estimators)
param_grid = [
    { 
        'reduce_dim__n_components' : [60],
        'clf__hidden_layer_sizes' : [(100,)],
        'clf__alpha' : [.0001, .001, .0005, .005],
        'clf' : [sklearn.neural_network.MLPClassifier()]  
    },
    {
        'reduce_dim__n_components' : [40,50,60],
        'clf__n_estimators' : [50, 100, 200],
        'clf' : [ensemble.AdaBoostClassifier()]        
    },
]
grid_search = sklearn.model_selection.GridSearchCV(pipe, param_grid=param_grid, n_jobs=4)
grid_search.fit(Xt, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('reduce_dim', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidde...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=4,
       param_grid=[{'clf__alpha': [0.0001, 0.001, 0.0005, 0.005], 'reduce_dim__n_components': [60], 'clf': [MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
   ...='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
  

In [108]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.951879699248
{'clf__alpha': 0.0005, 'reduce_dim__n_components': 60, 'clf': MLPClassifier(activation='relu', alpha=0.0005, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False), 'clf__hidden_layer_sizes': (100,)}


In [110]:
pipe = grid_search.best_estimator_

Pipeline(steps=[('reduce_dim', PCA(copy=True, iterated_power='auto', n_components=60, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', MLPClassifier(activation='relu', alpha=0.0005, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))])
